In [1]:
import os
import sys

# Get the current working directory
current_dir = os.getcwd()
# Add the ./src folder to the Python module search path
sys.path.append(os.path.join(current_dir, '..', 'src'))

from utils import *
from train_test import *
from collections import Counter
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
import xgboost as xgb

sns.set_style('ticks')

testing `train-test.py`.

In [2]:
feature_file = 'Mordred_reduced_features_96.npy'
features_file_2 =  'deepnose_features.npy'

# Mordred features
features = np.load(os.path.join(input_path, feature_file))
features_2 = np.load(os.path.join(input_path, features_file_2))

In [3]:
scaler = StandardScaler(with_mean=True, with_std=True)

# standardize Mordred
features = scaler.fit_transform(features)
# log standardize deepnose
epsilon = 1e-8 
features_2 = scaler.fit_transform(np.log(features_2 + epsilon))

In [4]:
# Convert DataFrame to a numpy array
features_array = features

# Create an imputer object with mean strategy, can change later!!!
imputer = SimpleImputer(strategy='mean')
# Impute missing values
imputed_features = imputer.fit_transform(features_array)

In [7]:
features_list = [imputed_features, features_2]
X_features, y_true = stacking_X_features(features_list, 'avg')

In [8]:
rf_best = {'n_estimators': 500, 'min_samples_split': 5, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}

In [9]:
models = ensemble_models(X_features, y_true, rf_best, type = 'rf', num_models = 10)

In [10]:
X_test, y_test_true = stacking_X_test_features(features_list,  X_features, "avg")

In [11]:
y_pred_avg = pred_mean(models, X_test)

In [12]:
# Calculate the correlation and R^2 for Random Forest
rf_corr = np.corrcoef(y_pred_avg, y_test_true)[0, 1]
rf_rmse = np.sqrt(mean_squared_error(np.array(y_test_true), y_pred_avg))

print(f"Random Forest - R: {rf_corr:.3f}")
print(f"Random Forest - RMSE: {rf_rmse:.3f}")


Random Forest - R: 0.710
Random Forest - RMSE: 0.123
